In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
class RatCatchingDataset(Dataset):
    def __init__(self, bot_grid, rat_grid, time_rem):
        # Convert numpy arrays to torch tensors
        self.bot_grid = torch.tensor(bot_grid, dtype=torch.float32)
        self.rat_grid = torch.tensor(rat_grid, dtype=torch.float32)
        self.time_rem = torch.tensor(time_rem, dtype=torch.float32)
        
    def __len__(self):
        return len(self.time_rem)
    
    def __getitem__(self, idx):
        # Combine bot and rat grids as input
        input_tensor = torch.stack([
            self.bot_grid[idx], 
            self.rat_grid[idx]
        ])
        return input_tensor, self.time_rem[idx]

In [ ]:
    def __init__(self, grid_size=30):
        super(RatCatchingNetwork, self).__init__()
        
        # Convolutional layers to extract features
        self.conv_layers = nn.Sequential(
            nn.Conv2d(2, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Calculate the flattened size after convolutions
        self.flattened_size = self._get_conv_output_size(grid_size)
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(self.flattened_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # Single output for time steps remaining
        )

In [ ]:
def main():
    data = np.load('seed_457.npz')
    bot_grid_array = data['bot_grid']
    rat_grid_array = data['rat_grid']
    time_rem_array = data['time_step_remaining']